In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import Adam
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader
import numpy as np

env = gym.make("CartPole-v1")

In [2]:
class Net(torch.nn.Module):
    def __init__(self, dims):
        super().__init__()
        self.layers = []
        for d in range(len(dims) - 1):
            self.layers += [Layer(dims[d], dims[d + 1])]

    def predict(self, x):
        goodness_per_label = []
        for label in range(2):  # 10 for mnist
            h = torch.tensor([x.tolist() + [label]])
            goodness = []
            for layer in self.layers:
                h = layer(h)
                goodness += [h.pow(2).mean(1)]
            goodness_per_label += [sum(goodness).unsqueeze(1)]
        goodness_per_label = torch.cat(goodness_per_label, 1)
        # print(goodness_per_label)
        return goodness_per_label.argmax(1)

    def train(self, x_pos, x_neg):
        h_pos, h_neg = x_pos, x_neg
        for i, layer in enumerate(self.layers):
            print('training layer', i, '...')
            h_pos, h_neg = layer.train(h_pos, h_neg)
    
    def is_good(self, x):
        goodness = []
        h = x.clone().detach()   # .tolist()
        for layer in self.layers:
            h = layer(h)
            # print(h)
            goodness += [h.pow(2).mean(1)]
        return sum(goodness)

In [3]:
class Layer(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
        super().__init__(in_features, out_features, bias, device, dtype)
        self.relu = torch.nn.ReLU()
        self.opt = Adam(self.parameters(), lr=0.0003)
        self.threshold = 2.0
        self.num_epochs = 1

    def forward(self, x):
        # print(x.shape)
        x_direction = x / (x.norm(2, 1, keepdim=True) + 1e-4) # normalization
        # print(x_direction)
        # torch.mm -> matrix multiplication
        # print(self.relu(torch.mm(x_direction, self.weight.T) + self.bias.unsqueeze(0)))
        out = self.relu(torch.mm(x_direction, self.weight.T) + self.bias.unsqueeze(0))
        if torch.any(torch.isnan(self.weight.data)).item():
            print("weight problem")
            # self.weight.data
        if torch.any(torch.isnan(out)).item():
            print("NaN problem")
            # print(out)
            # print(self.weight)
            # print(x.norm(2, 1, keepdim=True))
            # print(self.relu(torch.mm(x_direction, self.weight.T)))
            # raise ValueError
            out = torch.torch.nan_to_num(out)
            # self.parameters
        return out

    def train(self, x_pos, x_neg):
        for i in tqdm(range(self.num_epochs)):
            out_pos = self.forward(x_pos)
            out_neg = self.forward(x_neg)
            if torch.any(torch.isnan(out_pos)).item() or torch.any(torch.isnan(out_pos)).item():
                print("problem")
                # continue
            g_pos = out_pos.pow(2).mean(1)
            g_neg = out_neg.pow(2).mean(1)
            loss = torch.log(1 + torch.exp(torch.cat([-g_pos + self.threshold, g_neg - self.threshold]))).mean()
            # print(loss)
            # print(self.weight.data.pow(2).sum())
            if torch.any(torch.isnan(self.weight.data)).item():
                print("weight problem")
            if torch.any(torch.isnan(loss)).item():
                print("Loss problem")
            # if torch.isnan(loss):
            #     print("loss problem")
            # print(loss)
            self.opt.zero_grad()
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=2)
            self.opt.step()
        return self.forward(x_pos).detach(), self.forward(x_neg).detach()

In [4]:
from random import sample
from random import random
from statistics import mean
        
def get_posneg_data(epsilon=0.5, N=5, look_next = False, thresh=1):
    negative_data = []
    positive_data = []
    game_lens = []
    for i_episode in range(100):
        state, info = env.reset()
        game = []
        for t in count():
            s = random()
            if s > epsilon:
                action = env.action_space.sample()
            else:
                action = net.predict(torch.tensor(state)).item()
            observation, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            if terminated:
                next_state = None
                game.append(state.tolist() + [action])
            else:
                next_state = observation  
                game.append(state.tolist() + [action])
                if look_next and (net.is_good(torch.tensor([next_state.tolist() + [0]])) < thresh and
                                  net.is_good(torch.tensor([next_state.tolist() + [1]])) < thresh):
                    negative_data.append(state.tolist() + [action])
            
            state = next_state
            if done:
                break
          
        game_lens.append(len(game))
        if len(game) > N:
            negative_data += game[-N:]
            positive_data += game[:-N]
        else:
            negative_data += game
      
    print(mean(game_lens))
    pos_len = len(positive_data) 
    neg_len = len(negative_data)
    print(pos_len)
    print(neg_len)
    if pos_len > neg_len:
        positive_data = sample(positive_data, neg_len)
    else:
        negative_data = sample(negative_data, pos_len)
    
    return torch.tensor(positive_data), torch.tensor(negative_data)

In [7]:
net = Net([5, 100, 100]) # with mnist : 784, 500, 500
env = gym.make("CartPole-v1")

print("step 1")
for i in range(10):
    pos_data, neg_data = get_posneg_data(epsilon=0.6, N=2)
    net.train(pos_data, neg_data)
    
print("step 2")
for i in range(10):
    pos_data, neg_data = get_posneg_data(epsilon=0.4, N=2, look_next=True)
    net.train(pos_data, neg_data)
    
print("step 3")
for i in range(10):
    pos_data, neg_data = get_posneg_data(epsilon=0.2, N=2, look_next = True)
    net.train(pos_data, neg_data)
    
print("step 4")
for i in range(10):
    pos_data, neg_data = get_posneg_data(epsilon=0.1, N=2, look_next = True, thresh=0.5)
    net.train(pos_data, neg_data)
    
print("step 5")
for i in range(10):
    pos_data, neg_data = get_posneg_data(epsilon=0.1, N=2, look_next = True, thresh=0.5)
    net.train(pos_data, neg_data)

step 1
15.98
1398
200
training layer 0 ...


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 138.52it/s]


training layer 1 ...


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 136.78it/s]


15.74
1374
200
training layer 0 ...


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 40.77it/s]


21.38
1938
2238
training layer 0 ...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.06it/s]


training layer 1 ...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 42.37it/s]


In [ ]:
# print(pos_data[:40])
net.is_good(pos_data) # PROBLEM

In [ ]:
env = gym.make("CartPole-v1", render_mode="human")

for i in range(10):
    state, info = env.reset()
    rewards = []
    for t in count():
        action = net.predict(state).item()
        print(action)

        observation, reward, terminated, truncated, _ = env.step(action)
        rewards.append(reward)
        env.render()

        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = observation

        state = next_state

        if done:
            break
    print(sum(rewards))
            
print("Over")
env.close()

10.0
9.0
10.0
10.0
10.0
9.0
10.0
9.0
10.0
10.0
Over

Random:
    
27.0
18.0
15.0
16.0
13.0
22.0
11.0
23.0
41.0
20.0
Over

Model:
9.0
28.0
11.0
10.0
9.0
11.0
10.0
11.0
50.0
43.0
Over

Random
14.0
22.0
14.0
63.0
13.0
20.0
117.0
85.0
16.0
15.0
Over